<a href="https://colab.research.google.com/github/phdsasarun/Learn-Github/blob/main/Modi_CKD_0505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# การวิเคราะห์แนวโน้มการเกิดโรคไตเรื้อรัง (CKD) จากผู้ป่วยเบาหวาน
# ด้วยเทคนิค Machine Learning

# ติดตั้ง Libraries ที่จำเป็น
!pip install pandas numpy scikit-learn tensorflow matplotlib seaborn plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 1. โหลดข้อมูล
print("=== การโหลดข้อมูล ===")
# ใช้ gdown สำหรับดาวน์โหลดไฟล์จาก Google Drive
!pip install gdown
import gdown

# ดาวน์โหลดไฟล์จาก Google Drive
file_id = "12tU2nSdEy7OfbZtHr8pxUuuh00KxWkod"
url = f"https://drive.google.com/uc?id={file_id}"
output = "kidney_disease_dataset.csv"
gdown.download(url, output, quiet=False)

# โหลดข้อมูล
df = pd.read_csv(output)
print(f"ขนาดข้อมูล: {df.shape}")
print("\nข้อมูลตัวอย่าง 5 แถวแรก:")
print(df.head())

# 2. เลือกคอลัมน์ที่ต้องการ
print("\n=== การเลือกคอลัมน์ที่ต้องการ ===")
selected_columns = [
    'age',  # Age of the patient
    'bp',   # Blood pressure (mm/Hg)
    'al',   # Albumin in urine
    'bgr',  # Random blood glucose level (mg/dl)
    'sc',   # Serum creatinine (mg/dl)
    'hemo', # Hemoglobin level (gms)
    'pcv',  # Packed Cell Volume (ใช้แทน eGFR)
    'rc',   # Red Blood Cell Count (ใช้เป็นตัวแทนสำหรับ Cholesterol)
    'classification'  # Target variable
]

# ตรวจสอบคอลัมน์ที่มีในข้อมูล
print("คอลัมน์ทั้งหมดในข้อมูล:")
print(df.columns.tolist())

# เลือกคอลัมน์ที่มีอยู่จริง
available_columns = [col for col in selected_columns if col in df.columns]
print(f"\nคอลัมน์ที่เลือกใช้: {available_columns}")

df_selected = df[available_columns].copy()

# 3. การสำรวจข้อมูล (Exploratory Data Analysis)
print("\n=== การสำรวจข้อมูล ===")
print("ข้อมูลสถิติพื้นฐาน:")
print(df_selected.describe())

print("\nการตรวจสอบค่าที่หายไป:")
print(df_selected.isnull().sum())

print("\nการแจกแจงของ Target Variable:")
print(df_selected['classification'].value_counts())

# 4. การทำความสะอาดข้อมูล
print("\n=== การทำความสะอาดข้อมูล ===")

# แปลงค่า categorical เป็น numerical
label_encoders = {}
for column in df_selected.columns:
    if df_selected[column].dtype == 'object':
        if column != 'classification':
            le = LabelEncoder()
            df_selected[column] = le.fit_transform(df_selected[column].astype(str))
            label_encoders[column] = le

# จัดการค่าที่หายไป
df_selected = df_selected.fillna(df_selected.median())

# สร้าง CKD Risk Levels (5 ระดับ)
def create_ckd_risk_levels(row):
    """
    สร้างระดับความเสี่ยง CKD จากข้อมูลคลินิก
    เกณฑ์การวัดผล:
    - Age: >65 (ความเสี่ยงสูง)
    - BP: >140/90 (ความเสี่ยงสูง)
    - Serum Creatinine: >1.5 (ความเสี่ยงสูง)
    - Albumin: >2+ (ความเสี่ยงสูง)
    - Hemoglobin: <10 (ความเสี่ยงสูง)
    """
    risk_score = 0

    # Age factor
    if 'age' in row.index and row['age'] > 65:
        risk_score += 2
    elif 'age' in row.index and row['age'] > 45:
        risk_score += 1

    # Blood pressure factor
    if 'bp' in row.index and row['bp'] > 140:
        risk_score += 2
    elif 'bp' in row.index and row['bp'] > 120:
        risk_score += 1

    # Serum creatinine factor
    if 'sc' in row.index and row['sc'] > 1.5:
        risk_score += 2
    elif 'sc' in row.index and row['sc'] > 1.2:
        risk_score += 1

    # Albumin factor
    if 'al' in row.index and row['al'] > 2:
        risk_score += 2
    elif 'al' in row.index and row['al'] > 1:
        risk_score += 1

    # Hemoglobin factor
    if 'hemo' in row.index and row['hemo'] < 10:
        risk_score += 2
    elif 'hemo' in row.index and row['hemo'] < 12:
        risk_score += 1

    # Convert to 5-level classification
    if risk_score == 0:
        return 1  # No Disease
    elif risk_score <= 2:
        return 2  # Low Risk
    elif risk_score <= 4:
        return 3  # Moderate Risk
    elif risk_score <= 6:
        return 4  # High Risk
    else:
        return 5  # Severe Disease

# สร้าง target variable แบบ 5 ระดับ
if 'classification' in df_selected.columns:
    # ใช้ข้อมูล classification เดิมและปรับให้เป็น 5 ระดับ
    unique_classes = df_selected['classification'].unique()
    if len(unique_classes) == 2:  # ถ้าเป็น binary classification
        # แปลงเป็น 5 ระดับโดยใช้ clinical parameters
        df_selected['ckd_risk_level'] = df_selected.apply(create_ckd_risk_levels, axis=1)
    else:
        df_selected['ckd_risk_level'] = df_selected['classification']
else:
    df_selected['ckd_risk_level'] = df_selected.apply(create_ckd_risk_levels, axis=1)

# แสดงการแจกแจงของ CKD Risk Levels
print("\n=== การแจกแจงของระดับความเสี่ยง CKD ===")
risk_levels = {
    1: "No Disease (ไม่มีภาวะโรคไตเรื้อรัง)",
    2: "Low Risk (ความเสี่ยงต่ำ)",
    3: "Moderate Risk (ความเสี่ยงปานกลาง)",
    4: "High Risk (ความเสี่ยงสูง)",
    5: "Severe Disease (โรคไตเรื้อรังร้ายแรง)"
}

risk_distribution = df_selected['ckd_risk_level'].value_counts().sort_index()
print("จำนวนผู้ป่วยในแต่ละระดับ:")
for level, count in risk_distribution.items():
    print(f"ระดับ {level}: {risk_levels[level]} = {count} คน ({count/len(df_selected)*100:.1f}%)")

# 5. การแสดงผลด้วยกราฟ
print("\n=== การสร้างกราฟแสดงผล ===")

# กราฟแสดงการแจกแจงของ CKD Risk Levels
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('การแจกแจงระดับความเสี่ยง CKD', 'ความสัมพันธ์ระหว่างอายุและความเสี่ยง',
                   'ความสัมพันธ์ระหว่างความดันโลหิตและ Creatinine', 'การแจกแจงตามเพศ (ถ้ามี)'),
    specs=[[{"type": "bar"}, {"type": "box"}],
           [{"type": "scatter"}, {"type": "pie"}]]
)

# กราฎแท่งแสดงการแจกแจง
labels = [f"Level {i}: {risk_levels[i].split('(')[0].strip()}" for i in range(1, 6)]
values = [risk_distribution.get(i, 0) for i in range(1, 6)]

fig.add_trace(
    go.Bar(x=labels, y=values, name="จำนวนผู้ป่วย"),
    row=1, col=1
)

# Box plot สำหรับอายุและระดับความเสี่ยง
if 'age' in df_selected.columns:
    fig.add_trace(
        go.Box(y=df_selected['age'], x=df_selected['ckd_risk_level'], name="อายุ"),
        row=1, col=2
    )

# Scatter plot สำหรับความดันโลหิตและ Creatinine
if 'bp' in df_selected.columns and 'sc' in df_selected.columns:
    fig.add_trace(
        go.Scatter(x=df_selected['bp'], y=df_selected['sc'],
                  mode='markers',
                  marker=dict(color=df_selected['ckd_risk_level'], colorscale='Viridis'),
                  name="BP vs Creatinine"),
        row=2, col=1
    )

# Pie chart สำหรับการแจกแจง
fig.add_trace(
    go.Pie(labels=labels, values=values, name="สัดส่วน"),
    row=2, col=2
)

fig.update_layout(height=800, title_text="การวิเคราะห์ข้อมูล CKD Risk Analysis")
fig.show()

# 6. เตรียมข้อมูลสำหรับ Machine Learning
print("\n=== การเตรียมข้อมูลสำหรับ Machine Learning ===")

# แยก features และ target
feature_columns = [col for col in df_selected.columns if col not in ['classification', 'ckd_risk_level']]
X = df_selected[feature_columns]
y = df_selected['ckd_risk_level']

print(f"จำนวน Features: {len(feature_columns)}")
print(f"Features: {feature_columns}")
print(f"จำนวนตัวอย่าง: {len(X)}")

# แบ่งข้อมูลเป็น train และ test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"ข้อมูล Training: {X_train_scaled.shape}")
print(f"ข้อมูล Testing: {X_test_scaled.shape}")

# 7. Logistic Regression Model
print("\n=== Logistic Regression Model ===")

# สร้างและฝึกโมเดล Logistic Regression
lr_model = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

# ทำนายผล
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)

# ประเมินผล
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {lr_accuracy:.4f}")

print("\nClassification Report (Logistic Regression):")
print(classification_report(y_test, y_pred_lr,
                          target_names=[risk_levels[i] for i in sorted(y.unique())]))

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f"Level {i}" for i in sorted(y.unique())],
            yticklabels=[f"Level {i}" for i in sorted(y.unique())])
plt.title('Confusion Matrix - Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Feature Importance สำหรับ Logistic Regression
feature_importance_lr = np.abs(lr_model.coef_).mean(axis=0)
feature_df_lr = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importance_lr
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_df_lr, x='Importance', y='Feature')
plt.title('Feature Importance - Logistic Regression')
plt.show()

print("\nTop 5 สำคัญ Features (Logistic Regression):")
print(feature_df_lr.head())

# 8. Neural Network (Deep Learning) Model
print("\n=== Neural Network (Deep Learning) Model ===")

# แปลง target เป็น categorical สำหรับ Neural Network
num_classes = len(y.unique())
y_train_cat = tf.keras.utils.to_categorical(y_train - 1, num_classes)  # -1 เพราะ class เริ่มจาก 1
y_test_cat = tf.keras.utils.to_categorical(y_test - 1, num_classes)

# สร้าง Neural Network Model
def create_nn_model(input_dim, num_classes):
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),

        Dense(16, activation='relu'),
        Dropout(0.2),

        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# สร้างโมเดล
nn_model = create_nn_model(X_train_scaled.shape[1], num_classes)
print("Neural Network Architecture:")
nn_model.summary()

# Callbacks สำหรับการฝึก
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)

# ฝึกโมเดล
history = nn_model.fit(
    X_train_scaled, y_train_cat,
    validation_data=(X_test_scaled, y_test_cat),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# ทำนายผล
y_pred_nn = nn_model.predict(X_test_scaled)
y_pred_nn_classes = np.argmax(y_pred_nn, axis=1) + 1  # +1 เพราะ class เริ่มจาก 1

# ประเมินผล
nn_accuracy = accuracy_score(y_test, y_pred_nn_classes)
print(f"\nNeural Network Accuracy: {nn_accuracy:.4f}")

print("\nClassification Report (Neural Network):")
print(classification_report(y_test, y_pred_nn_classes,
                          target_names=[risk_levels[i] for i in sorted(y.unique())]))

# Confusion Matrix สำหรับ Neural Network
plt.figure(figsize=(10, 8))
cm_nn = confusion_matrix(y_test, y_pred_nn_classes)
sns.heatmap(cm_nn, annot=True, fmt='d', cmap='Greens',
            xticklabels=[f"Level {i}" for i in sorted(y.unique())],
            yticklabels=[f"Level {i}" for i in sorted(y.unique())])
plt.title('Confusion Matrix - Neural Network')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# แสดงกราฟการฝึก
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 3)
plt.bar(['Logistic Regression', 'Neural Network'], [lr_accuracy, nn_accuracy])
plt.title('Model Comparison')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
for i, v in enumerate([lr_accuracy, nn_accuracy]):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.show()

# 9. การเปรียบเทียบผลลัพธ์
print("\n=== การเปรียบเทียบผลลัพธ์ ===")
results_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Neural Network'],
    'Accuracy': [lr_accuracy, nn_accuracy],
    'Training_Time': ['Fast', 'Moderate'],
    'Interpretability': ['High', 'Low']
})

print(results_comparison)

# 10. เกณฑ์การวัดผลในแต่ละ Item
print("\n=== เกณฑ์การวัดผลในแต่ละ Item ===")
criteria = {
    'Age (อายุ)': {
        'ปกติ': '< 45 ปี',
        'เสี่ยงปานกลาง': '45-65 ปี',
        'เสี่ยงสูง': '> 65 ปี'
    },
    'Blood Pressure (ความดันโลหิต)': {
        'ปกติ': '< 120/80 mmHg',
        'เสี่ยงปานกลาง': '120-140/80-90 mmHg',
        'เสี่ยงสูง': '> 140/90 mmHg'
    },
    'Serum Creatinine (ครีอาตินิน)': {
        'ปกติ': '< 1.2 mg/dl',
        'เสี่ยงปานกลาง': '1.2-1.5 mg/dl',
        'เสี่ยงสูง': '> 1.5 mg/dl'
    },
    'Albumin in Urine (โปรตีนในปัสสาวะ)': {
        'ปกติ': 'Negative/Trace',
        'เสี่ยงปานกลาง': '1+',
        'เสี่ยงสูง': '2+ ขึ้นไป'
    },
    'Hemoglobin (ฮีโมโกลบิน)': {
        'ปกติ': '> 12 g/dl',
        'เสี่ยงปานกลาง': '10-12 g/dl',
        'เสี่ยงสูง': '< 10 g/dl'
    },
    'Blood Glucose (ระดับน้ำตาลในเลือด)': {
        'ปกติ': '< 100 mg/dl',
        'เสี่ยงปานกลาง': '100-125 mg/dl',
        'เสี่ยงสูง': '> 125 mg/dl'
    }
}

for item, ranges in criteria.items():
    print(f"\n{item}:")
    for level, range_val in ranges.items():
        print(f"  - {level}: {range_val}")

# 11. สรุปผลการวิจัย
print("\n=== สรุปผลการวิจัย ===")
print("1. การแจกแจงของผู้ป่วย:")
for level, count in risk_distribution.items():
    percentage = count/len(df_selected)*100
    print(f"   - {risk_levels[level]}: {count} คน ({percentage:.1f}%)")

print(f"\n2. ประสิทธิภาพของโมเดล:")
print(f"   - Logistic Regression: {lr_accuracy:.1%}")
print(f"   - Neural Network: {nn_accuracy:.1%}")

if nn_accuracy > lr_accuracy:
    print(f"\n3. Neural Network มีประสิทธิภาพสูงกว่า Logistic Regression {(nn_accuracy-lr_accuracy)*100:.1f}%")
else:
    print(f"\n3. Logistic Regression มีประสิทธิภาพสูงกว่า Neural Network {(lr_accuracy-nn_accuracy)*100:.1f}%")

print("\n4. ปัจจัยสำคัญในการทำนาย:")
print("   Top 3 Features สำคัญ:")
for i, (_, row) in enumerate(feature_df_lr.head(3).iterrows()):
    print(f"   {i+1}. {row['Feature']}: {row['Importance']:.4f}")

print("\n=== เสร็จสิ้นการวิเคราะห์ ===")

# 12. บันทึกผลลัพธ์
print("\n=== การบันทึกผลลัพธ์ ===")

# บันทึกผลการทำนาย
predictions_df = pd.DataFrame({
    'Actual': y_test,
    'Logistic_Regression': y_pred_lr,
    'Neural_Network': y_pred_nn_classes
})

# บันทึกลงไฟล์
predictions_df.to_csv('ckd_predictions.csv', index=False)
results_comparison.to_csv('model_comparison.csv', index=False)
feature_df_lr.to_csv('feature_importance.csv', index=False)

print("บันทึกไฟล์เรียบร้อย:")
print("- ckd_predictions.csv: ผลการทำนายของทั้งสองโมเดล")
print("- model_comparison.csv: การเปรียบเทียบประสิทธิภาพโมเดล")
print("- feature_importance.csv: ความสำคัญของ Features")

print("\n🎉 โครงงานวิจัยเสร็จสมบูรณ์! 🎉")

=== การโหลดข้อมูล ===


Downloading...
From: https://drive.google.com/uc?id=12tU2nSdEy7OfbZtHr8pxUuuh00KxWkod
To: /content/kidney_disease_dataset.csv
100%|██████████| 5.48M/5.48M [00:00<00:00, 19.3MB/s]


ขนาดข้อมูล: (20538, 43)

ข้อมูลตัวอย่าง 5 แถวแรก:
   Age of the patient  Blood pressure (mm/Hg)  Specific gravity of urine  \
0                  54                     167                      1.023   
1                  42                     127                      1.023   
2                  38                     148                      1.016   
3                   7                      98                      1.017   
4                  67                     174                      1.015   

   Albumin in urine  Sugar in urine Red blood cells in urine  \
0                 1               4                   normal   
1                 3               2                   normal   
2                 0               0                 abnormal   
3                 4               0                 abnormal   
4                 1               1                   normal   

  Pus cells in urine Pus cell clumps in urine Bacteria in urine  \
0           abnormal              not pre

ValueError: Cannot describe a DataFrame without columns